In [17]:
%matplotlib notebook
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, RFECV, f_classif
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from functools import reduce
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from imblearn.over_sampling import RandomOverSampler
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve
#from sklearn.metrics import make_scorer
#from sklearn.metrics import accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt
import imblearn
from imblearn.over_sampling import SMOTENC, SMOTEN
from collections import Counter
import tensorflow as tf
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import cross_val_score
from keras.metrics import accuracy
from statistics import mean

In [2]:
#data=pd.read_csv(filepath_or_buffer='generated_csv/hrv_completed.csv')
data=pd.read_csv(filepath_or_buffer='C:/Users/catar/Desktop/TUe/Conferences/PETRA_2023/Code_and_data/generated_csv/hrv_completed.csv')

pd.set_option('display.max_columns', None)
    
data.participant=data.participant.astype('category').cat.codes #turn the participant column into categorical numbers (instead of keep the string with letters+numbers because it would raise an error later on)
scaler=MinMaxScaler(feature_range=(1, 3))
cols_to_normalize=['arousal','arousal_groups','valence','valence_groups','arousal_valence_4_grouping','wasted','wasted_groups','location','ts','activity','hr_avg','hr_min','hr_max','hr_std','rmssd','sdrr','delta_time_min','hrv_baseline','rmssd_compared_hrv_baseline'] #normalize all columns

In [3]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True): # returns a Pandas DataFrame of series framed for supervised learning
	
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)

	cols, names = list(), list()
	#columns=['participant','arousal','arousal_groups','valence','valence_groups','arousal_valence_4_grouping','wasted','wasted_groups','location','hour_min','ts','activity','hr_avg','hr_min','hr_max','hr_std','rmssd','sdrr','delta_time_min']

	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		"""shift() -> this function is used to create copies of columns that are pushed forward (rows of NaN values added to the front) or pulled back (rows of NaN values added to the end). This is the behavior required to create
		columns of lag observations as well as columns of forecast observations for a time series dataset in a supervised learning format."""
		names += [('var%s(t-%d)' % (j, i)) for j in range(n_vars)]
	
    # forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%s(t)' % (j)) for j in range(n_vars)]
		else:
			names += [('var%s(t+%d)' % (j, i)) for j in range(n_vars)]
	
    # put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	
    # drop rows with NaN values
	#if dropnan:
	#	agg.dropna(inplace=True)
	return agg

Using SMOTE as the oversampling technique; Cross-Validation (k=5); Hyperparameter tuning w/ Randomized Search CV; Iterative Features Selection.

In [52]:
results = open('C:/Users/catar/Desktop/TUe/Conferences/PETRA_2023/Code_and_data/Best_scenarios/sensor_location_action_report/classifiers_standardization/results_standardization.csv', 'w')
results.write('Classifier,Avg_AUC,Avg_Acc,Number_of_Features,Features,AUC_each_fold,Acc_each_fold,TPR_multiclass_each_fold,FPR_multiclass_each_fold,Best_Hyperparameters\n') #columns names
results.close()

#Decision tree does not need normalization
data_copy=data.copy()
data_copy[cols_to_normalize]=scaler.fit_transform(data[cols_to_normalize])
values=data_copy.values
#values=data.values

# convert series to supervised learning
reframed = series_to_supervised(values, 1, 1) #shift to get the time features

"""
Reframed index meaning:
0: participant(t-1); 1:arousal(t-1); 2:arousal_groups(t-1); 3:valence(t-1); 4:valence_groups(t-1); 5: arousal_valence_4_grouping(t-1); 6: wasted(t-1); 7: wasted_groups(t-1); 8: location(t-1) 
9: hour_min(t-1); 10: ts(t-1); 11: activity(t-1); 12: hr_avg(t-1); 13: hr_min(t-1); 14: hr_max(t-1); 15: hr_std(t-1); 16: rmssd(t-1); 17: sdrr(t-1); 18: delta_time_min(t-1); 19: hrv_baseline(t-1); 20: rmssd_compared_hrv_baseline(t-1)
21: participant(t); 22:arousal(t); 23:arousal_groups(t); 24:valence(t); 25:valence_groups(t); 26: arousal_valence_4_grouping(t); 27: wasted(t); 28: wasted_groups(t); 29: location(t) 30: hour_min(t); 31: ts(t); 
32: activity(t); 33: hr_avg(t); 34: hr_min(t); 35: hr_max(t); 36: hr_std(t); 37: rmssd(t); 38: sdrr(t); 39: delta_time_min(t); 40: hrv_baseline(t); 41: rmssd_compared_hrv_baseline(t)
"""

reframed.drop(reframed.columns[[9,18,24,26,30,40]], axis=1, inplace=True) #18 -> delta_time_min(t-1); 24 -> valence(t); 26 -> arousal_valence_4_grouping.(t) --> makes no sense to keep; 9:hour_min(t-1) + 30 hour_min(t-1) -> we already have ts; it won't add new information

"""
reframed index meaning after the drop:
0: participant(t-1); 1:arousal(t-1); 2:arousal_groups(t-1); 3:valence(t-1); 4:valence_groups(t-1); 5: arousal_valence_4_grouping(t-1); 6: wasted(t-1); 7: wasted_groups(t-1); 8: location(t-1); 9: ts(t-1); 
10: activity(t-1); 11: hr_avg(t-1); 12: hr_min(t-1); 13: hr_max(t-1); 14: hr_std(t-1); 15: rmssd(t-1); 16: sdrr(t-1); 17: hrv_baseline(t-1); 18: rmssd_compared_hrv_baseline(t-1)
19: participant(t); 20:arousal(t); 21:arousal_groups(t); 22:valence_groups(t); 23: wasted(t); 24: wasted_groups(t); 25: location(t); 26: ts(t); 27: activity(t); 28: hr_avg(t); 29: hr_min(t); 
30: hr_max(t); 31: hr_std(t); 32: rmssd(t); 33: sdrr(t); 34: delta_time_min(t); 35: rmssd_compared_hrv_baseline(t)
"""

print(reframed)
#make the reframed only have rows with data from one participant (and not rows that have 2 participants data)
X=[]
y=[]
for index, row in reframed.iterrows():
    row_list=row.to_list()
    if row_list[0]==row_list[19]: #if the data belongs to the same participant
        X.append(row_list[1:19]+row_list[20:22]+row_list[23:])
        y.append(row_list[22])


"""
X:
arousal(t-1); arousal_groups(t-1); valence(t-1); valence_groups(t-1); arousal_valence_4_grouping(t-1); wasted(t-1); wasted_groups(t-1); location(t-1); ts(t-1); activity(t-1); hr_avg(t-1); hr_min(t-1); 
hr_max(t-1); hr_std(t-1); rmssd(t-1); sdrr(t-1); hrv_baseline(t-1); rmssd_compared_hrv_baseline(t-1)
arousal(t); arousal_groups(t); wasted(t); wasted_groups(t); location(t); ts(t); activity(t); hr_avg(t); hr_min(t); hr_max(t); hr_std(t); rmssd(t); sdrr(t); delta_time_min(t); rmssd_compared_hrv_baseline(t)
"""

X=np.array(X)
y=np.array(y)
y=y.ravel()

cols=['arousal(t-1)', 'arousal_groups(t-1)', 'valence(t-1)', 'valence_groups(t-1)', 'arousal_valence_4_grouping(t-1)', 'wasted(t-1)', 'wasted_groups(t-1)', 'location(t-1)', 'ts(t-1)', 'activity(t-1)', 
'hr_avg(t-1)', 'hr_min(t-1)', 'hr_max(t-1)', 'hr_std(t-1)', 'rmssd(t-1)', 'sdrr(t-1)', 'hrv_baseline(t-1)', 'rmssd_compared_hrv_baseline(t-1)', 'arousal(t)', 'arousal_groups(t)', 
'wasted(t)', 'wasted_groups(t)', 'location(t)', 'ts(t)', 'activity(t)', 'hr_avg(t)', 'hr_min(t)', 'hr_max(t)', 'hr_std(t)', 'rmssd(t)', 'sdrr(t)', 'delta_time_min(t)', 'rmssd_compared_hrv_baseline(t)']

cat_cols=['arousal(t-1)','arousal_groups(t-1)','valence(t-1)','valence_groups(t-1)','arousal_valence_4_grouping(t-1)','wasted(t-1)','wasted_groups(t-1)','location(t-1)','activity(t-1)','arousal(t)',
'arousal_groups(t)','wasted(t)','wasted_groups(t)','location(t)','activity(t)']

def classifiers_definition(X_train):
    classifiers = [
        ('KNearest Neighbors', KNeighborsClassifier(), {
                'weights': ['uniform', 'distance'],
                'algorithm': ['auto']}), #'ball_tree', 'kd_tree', 'brute'

        ('Decision Tree', DecisionTreeClassifier(), {
            'criterion': ['gini', 'entropy'],
            'max_depth': [2,3,4,5],
            'min_samples_split': np.linspace(0.1, 1.0, 10, endpoint=True),
            'min_samples_leaf': np.linspace(0.1, 0.5, 5, endpoint=True),
            'max_features': list(range(1, X_train.shape[1])),
            'random_state': [0]}),

        ('Random Forest', RandomForestClassifier(), {
            'criterion': ['gini', 'entropy'],
            'max_depth': [2,3,4,5],
            'min_samples_split': np.linspace(0.1, 1.0, 10, endpoint=True),
            'min_samples_leaf': np.linspace(0.1, 0.5, 5, endpoint=True),
            'max_features': list(range(1, X_train.shape[1])),
            'random_state': [0,42]}),

        ('Neural Network', MLPClassifier(max_iter=1000), {
                'alpha': 10.0 ** -np.arange(1, 7),
                'solver': ['adam'],
                'random_state': [0,42]}),

        ('AdaBoost', AdaBoostClassifier(), {
                'algorithm': ['SAMME', 'SAMME.R'],
                'random_state': [0,42]}),

        ('Naive Bayes', GaussianNB(), {}),

        ('QDA', QuadraticDiscriminantAnalysis(), {}),
        
        ('RBF SVM', SVC(probability=True), {
                'random_state': [0,42]})
        ]

    return classifiers

def score_model(cols, cat_cols):
    #number_of_features=[]
    #accuracies_valence_groups_time, number_of_features_valence_groups_time, val_score_folds = [],[],[]

    cv= StratifiedKFold(n_splits=5)
    KNN_acc,DT_acc,RF_acc,NN_acc,AB_acc,NB_acc,QDA_acc,SVM_acc=[],[],[],[],[],[],[],[] #each list will hold the test scores, for each classifier, for the different amount of features.
    KNN_auc,DT_auc,RF_auc,NN_auc,AB_auc,NB_auc,QDA_auc,SVM_auc=[],[],[],[],[],[],[],[]

    # iterate through num features for feature selection
    for k in range(2, len(cols)+1):
        KNN,DT,RF,NN,AB,NB,QDA,SVM=[],[],[],[],[],[],[],[] #each of these lists will hold the validation scores in each fold (cv) for each classifier for a certain amount of features.
        number_of_features=[]

        indexes=[]
        print('--- K: {}'.format(k))
        selector = SelectKBest(f_classif, k=k)
        selector.fit(X, y)
        X_new = selector.transform(X)
        mask = selector.get_support(indices=True)
        features = ''
        for i,index in enumerate(mask):
            features += cols[index] + ';'
            if cols[index] in cat_cols:
                indexes.append(i)

        #train test split -> we won't do it so we use the whole data for the cross validation
        #X_train, X_test, y_train, y_test = train_test_split(X_new, y, stratify=y, shuffle=True, test_size=.3, train_size=.7, random_state=0)

        #definition of the oversampling techniques; however, these will only really happen during each fold of the cross validation
        if k!=len(indexes): #this means we don't just have categorical features and, therefore, smotenc is appropriate
            oversample=SMOTENC(indexes, random_state=0)
        else: #in the cases where we only have categorical features (that were selected) we must use SMOTEN
            oversample=SMOTEN()
        
        fold_number=1

        for train_fold_index, test_fold_index in cv.split(X_new, y):
            print('fold_number:', fold_number)
            X_train_fold, y_train_fold = X_new[train_fold_index], y[train_fold_index]
            X_test_fold, y_test_fold = X_new[test_fold_index], y[test_fold_index]

            X_over_train, y_over_train= oversample.fit_resample(X_train_fold, y_train_fold)

            classifiers=classifiers_definition(X_over_train)

            for clf in classifiers:
                if bool(clf[2]):
                    n_iter = reduce((lambda _x, _y: _x * _y), [len(clf[2][k]) for k in clf[2]]) #number of scenarios in each algorithm
                else:
                    n_iter = 1

                print(clf[0])
                print('Grid, {}'.format(n_iter))
                
                if fold_number==1:
                    tuned_clf = RandomizedSearchCV(clf[1], clf[2], n_jobs=2, return_train_score=True) #when cv is not specified it takes the value of 5.
                
                #fit the model
                tuned_clf.fit(X_over_train, y_over_train)
                y_pred_proba=tuned_clf.predict_proba(X_test_fold)[::,1] #define metrics; to create a ROC curve we need to get the TPR and FPR
                #print('y_pred_proba',y_pred_proba)
                fpr_multiclass,tpr_multiclass,auc_multiclass=[],[],[]

                for iteration in range(3):
                    y_test_bin=[]
                    for element in y_test_fold:
                        if (iteration==0 and element==2) or (iteration==0 and element==3):
                            y_test_bin.append(0)
                        elif iteration==0 and element==1:
                            y_test_bin.append(1)
                        elif (iteration==1 and element==1) or (iteration==1 and element==3):
                            y_test_bin.append(0)
                        elif (iteration==1 and element==2):
                            y_test_bin.append(1)
                        elif (iteration==2 and element==1) or (iteration==2 and element==2):
                            y_test_bin.append(0)
                        elif (iteration==2 and element==3):
                            y_test_bin.append(1)

                    fpr,tpr,_=metrics.roc_curve(y_test_bin, y_pred_proba)
                    auc=metrics.roc_auc_score(y_test_bin, y_pred_proba)

                    fpr_multiclass.append(fpr)
                    tpr_multiclass.append(tpr)
                    auc_multiclass.append(auc)

                    if iteration==0:
                        auc_multiclass_sum=auc
                    else:
                        auc_multiclass_sum+=auc


                auc_multiclass_avg=auc_multiclass_sum/3

                print('AUC_avg:',auc_multiclass_avg)

                accuracy=accuracy_score(y_test_fold, tuned_clf.predict(X_test_fold))
                #tpr_sum,fpr_sum,auc_sum,accuracy_sum=0,0,0,0

                if clf[0]=='KNearest Neighbors':
                    KNN.append((tpr_multiclass,fpr_multiclass,auc_multiclass,auc_multiclass_avg,accuracy))

                    if fold_number==5:
                        accuracy_each_fold,tpr_each_fold,fpr_each_fold,auc_each_fold=[],[],[],[]
                        for index,final_results in enumerate(KNN):
                            accuracy_each_fold.append(final_results[4])
                            tpr_each_fold.append(final_results[0])
                            fpr_each_fold.append(final_results[1])
                            auc_each_fold.append(final_results[2])
                            if index==0:
                                auc_sum=final_results[3]
                                accuracy_sum=final_results[4]
                            else:
                                auc_sum+=final_results[3]
                                accuracy_sum+=final_results[4]

                        results = open('C:/Users/catar/Desktop/TUe/Conferences/PETRA_2023/Code_and_data/Best_scenarios/sensor_location_action_report/classifiers_standardization/results_standardization.csv', 'a')
                        results.write('{},{},{},{},{},{},{},{},{},{}\n'.format(clf[0], auc_sum/5, accuracy_sum/5, k, features[:-1],auc_each_fold, accuracy_each_fold, tpr_each_fold, fpr_each_fold,tuned_clf.best_params_))
                        results.close()
                        KNN_acc.append(accuracy_sum/5)
                        KNN_auc.append(accuracy_sum/5)
                            
                        #KNN,KNN_tuned_clf=classifiers_scores(clf,val_score,tuned_clf,KNN,DT,RF,NN,AB,NB,QDA,SVM)

                elif clf[0]=='Decision Tree':
                    DT.append((tpr_multiclass,fpr_multiclass,auc_multiclass,auc_multiclass_avg,accuracy))

                    if fold_number==5:
                        accuracy_each_fold,tpr_each_fold,fpr_each_fold,auc_each_fold=[],[],[],[]
                        for index,final_results in enumerate(DT):
                            accuracy_each_fold.append(final_results[4])
                            tpr_each_fold.append(final_results[0])
                            fpr_each_fold.append(final_results[1])
                            auc_each_fold.append(final_results[2])
                            if index==0:
                                auc_sum=final_results[3]
                                accuracy_sum=final_results[4]
                            else:
                                auc_sum+=final_results[3]
                                accuracy_sum+=final_results[4]

                        results = open('C:/Users/catar/Desktop/TUe/Conferences/PETRA_2023/Code_and_data/Best_scenarios/sensor_location_action_report/classifiers_standardization/results_standardization.csv', 'a')
                        results.write('{},{},{},{},{},{},{},{},{},{}\n'.format(clf[0], auc_sum/5, accuracy_sum/5, k, features[:-1],auc_each_fold, accuracy_each_fold, tpr_each_fold, fpr_each_fold,tuned_clf.best_params_))
                        results.close()
                        DT_acc.append(accuracy_sum/5)
                        DT_auc.append(accuracy_sum/5)
                    
                    #DT,DT_tuned_clf=classifiers_scores(clf,val_score,tuned_clf,KNN,DT,RF,NN,AB,NB,QDA,SVM)
                elif clf[0]=='Random Forest':
                    RF.append((tpr_multiclass,fpr_multiclass,auc_multiclass,auc_multiclass_avg,accuracy))

                    if fold_number==5:
                        accuracy_each_fold,tpr_each_fold,fpr_each_fold,auc_each_fold=[],[],[],[]
                        for index,final_results in enumerate(RF):
                            accuracy_each_fold.append(final_results[4])
                            tpr_each_fold.append(final_results[0])
                            fpr_each_fold.append(final_results[1])
                            auc_each_fold.append(final_results[2])
                            if index==0:
                                auc_sum=final_results[3]
                                accuracy_sum=final_results[4]
                            else:
                                auc_sum+=final_results[3]
                                accuracy_sum+=final_results[4]

                        results = open('C:/Users/catar/Desktop/TUe/Conferences/PETRA_2023/Code_and_data/Best_scenarios/sensor_location_action_report/classifiers_standardization/results_standardization.csv', 'a')
                        results.write('{},{},{},{},{},{},{},{},{},{}\n'.format(clf[0], auc_sum/5, accuracy_sum/5, k, features[:-1],auc_each_fold, accuracy_each_fold, tpr_each_fold, fpr_each_fold,tuned_clf.best_params_))
                        results.close()
                        RF_acc.append(accuracy_sum/5)
                        RF_auc.append(accuracy_sum/5)

                    #RF,RF_tuned_clf=classifiers_scores(clf,val_score,tuned_clf,KNN,DT,RF,NN,AB,NB,QDA,SVM)
                elif clf[0]=='Neural Network':
                    NN.append((tpr_multiclass,fpr_multiclass,auc_multiclass,auc_multiclass_avg,accuracy))

                    if fold_number==5:
                        accuracy_each_fold,tpr_each_fold,fpr_each_fold,auc_each_fold=[],[],[],[]
                        for index,final_results in enumerate(NN):
                            accuracy_each_fold.append(final_results[4])
                            tpr_each_fold.append(final_results[0])
                            fpr_each_fold.append(final_results[1])
                            auc_each_fold.append(final_results[2])
                            if index==0:
                                auc_sum=final_results[3]
                                accuracy_sum=final_results[4]
                            else:
                                auc_sum+=final_results[3]
                                accuracy_sum+=final_results[4]

                        results = open('C:/Users/catar/Desktop/TUe/Conferences/PETRA_2023/Code_and_data/Best_scenarios/sensor_location_action_report/classifiers_standardization/results_standardization.csv', 'a')
                        results.write('{},{},{},{},{},{},{},{},{},{}\n'.format(clf[0], auc_sum/5, accuracy_sum/5, k, features[:-1],auc_each_fold, accuracy_each_fold, tpr_each_fold, fpr_each_fold,tuned_clf.best_params_))
                        results.close()
                        NN_acc.append(accuracy_sum/5)
                        NN_auc.append(accuracy_sum/5)

                    #NN,NN_tuned_clf=classifiers_scores(clf,val_score,tuned_clf,KNN,DT,RF,NN,AB,NB,QDA,SVM)
                elif clf[0]=='AdaBoost':
                    AB.append((tpr_multiclass,fpr_multiclass,auc_multiclass,auc_multiclass_avg,accuracy))

                    if fold_number==5:
                        accuracy_each_fold,tpr_each_fold,fpr_each_fold,auc_each_fold=[],[],[],[]
                        for index,final_results in enumerate(AB):
                            accuracy_each_fold.append(final_results[4])
                            tpr_each_fold.append(final_results[0])
                            fpr_each_fold.append(final_results[1])
                            auc_each_fold.append(final_results[2])
                            if index==0:
                                auc_sum=final_results[3]
                                accuracy_sum=final_results[4]
                            else:
                                auc_sum+=final_results[3]
                                accuracy_sum+=final_results[4]

                        results = open('C:/Users/catar/Desktop/TUe/Conferences/PETRA_2023/Code_and_data/Best_scenarios/sensor_location_action_report/classifiers_standardization/results_standardization.csv', 'a')
                        results.write('{},{},{},{},{},{},{},{},{},{}\n'.format(clf[0], auc_sum/5, accuracy_sum/5, k, features[:-1],auc_each_fold, accuracy_each_fold, tpr_each_fold, fpr_each_fold,tuned_clf.best_params_))
                        results.close()
                        AB_acc.append(accuracy_sum/5)
                        AB_auc.append(accuracy_sum/5)
                    
                    #AB,AB_tuned_clf=classifiers_scores(clf,val_score,tuned_clf,KNN,DT,RF,NN,AB,NB,QDA,SVM)
                elif clf[0]=='Naive Bayes':
                    NB.append((tpr_multiclass,fpr_multiclass,auc_multiclass,auc_multiclass_avg,accuracy))

                    if fold_number==5:
                        accuracy_each_fold,tpr_each_fold,fpr_each_fold,auc_each_fold=[],[],[],[]
                        for index,final_results in enumerate(NB):
                            accuracy_each_fold.append(final_results[4])
                            tpr_each_fold.append(final_results[0])
                            fpr_each_fold.append(final_results[1])
                            auc_each_fold.append(final_results[2])
                            if index==0:
                                auc_sum=final_results[3]
                                accuracy_sum=final_results[4]
                            else:
                                auc_sum+=final_results[3]
                                accuracy_sum+=final_results[4]

                        results = open('C:/Users/catar/Desktop/TUe/Conferences/PETRA_2023/Code_and_data/Best_scenarios/sensor_location_action_report/classifiers_standardization/results_standardization.csv', 'a')
                        results.write('{},{},{},{},{},{},{},{},{},{}\n'.format(clf[0], auc_sum/5, accuracy_sum/5, k, features[:-1],auc_each_fold, accuracy_each_fold, tpr_each_fold, fpr_each_fold,tuned_clf.best_params_))
                        results.close()
                        NB_acc.append(accuracy_sum/5)
                        NB_auc.append(accuracy_sum/5)

                    #NB,NB_tuned_clf=classifiers_scores(clf,val_score,tuned_clf,KNN,DT,RF,NN,AB,NB,QDA,SVM)
                elif clf[0]=='QDA':
                    QDA.append((tpr_multiclass,fpr_multiclass,auc_multiclass,auc_multiclass_avg,accuracy))

                    if fold_number==5:
                        accuracy_each_fold,tpr_each_fold,fpr_each_fold,auc_each_fold=[],[],[],[]
                        for index,final_results in enumerate(QDA):
                            accuracy_each_fold.append(final_results[4])
                            tpr_each_fold.append(final_results[0])
                            fpr_each_fold.append(final_results[1])
                            auc_each_fold.append(final_results[2])
                            if index==0:
                                auc_sum=final_results[3]
                                accuracy_sum=final_results[4]
                            else:
                                auc_sum+=final_results[3]
                                accuracy_sum+=final_results[4]

                        results = open('C:/Users/catar/Desktop/TUe/Conferences/PETRA_2023/Code_and_data/Best_scenarios/sensor_location_action_report/classifiers_standardization/results_standardization.csv', 'a')
                        results.write('{},{},{},{},{},{},{},{},{},{}\n'.format(clf[0], auc_sum/5, accuracy_sum/5, k, features[:-1],auc_each_fold, accuracy_each_fold, tpr_each_fold, fpr_each_fold,tuned_clf.best_params_))
                        results.close()
                        QDA_acc.append(accuracy_sum/5)
                        QDA_auc.append(accuracy_sum/5)

                    #QDA,QDA_tuned_clf=classifiers_scores(clf,val_score,tuned_clf,KNN,DT,RF,NN,AB,NB,QDA,SVM)
                elif clf[0]=='RBF SVM':
                    SVM.append((tpr_multiclass,fpr_multiclass,auc_multiclass,auc_multiclass_avg,accuracy))

                    if fold_number==5:
                        accuracy_each_fold,tpr_each_fold,fpr_each_fold,auc_each_fold=[],[],[],[]
                        for index,final_results in enumerate(SVM):
                            accuracy_each_fold.append(final_results[4])
                            tpr_each_fold.append(final_results[0])
                            fpr_each_fold.append(final_results[1])
                            auc_each_fold.append(final_results[2])
                            if index==0:
                                auc_sum=final_results[3]
                                accuracy_sum=final_results[4]
                            else:
                                auc_sum+=final_results[3]
                                accuracy_sum+=final_results[4]

                        results = open('C:/Users/catar/Desktop/TUe/Conferences/PETRA_2023/Code_and_data/Best_scenarios/sensor_location_action_report/classifiers_standardization/results_standardization.csv', 'a')
                        results.write('{},{},{},{},{},{},{},{},{},{}\n'.format(clf[0], auc_sum/5, accuracy_sum/5, k, features[:-1],auc_each_fold, accuracy_each_fold, tpr_each_fold, fpr_each_fold,tuned_clf.best_params_))
                        results.close()
                        SVM_acc.append(accuracy_sum/5)
                        SVM_auc.append(accuracy_sum/5)
                    #SVM,SVM_tuned_clf=classifiers_scores(clf,val_score,tuned_clf,KNN,DT,RF,NN,AB,NB,QDA,SVM)

            fold_number+=1
        number_of_features.append(k)

    return KNN_acc,DT_acc,RF_acc,NN_acc,AB_acc,NB_acc,QDA_acc,SVM_acc,KNN_auc,DT_auc,RF_auc,NN_auc,AB_auc,NB_auc,QDA_auc,SVM_auc,number_of_features

KNN_acc,DT_acc,RF_acc,NN_acc,AB_acc,NB_acc,QDA_acc,SVM_acc,KNN_auc,DT_auc,RF_auc,NN_auc,AB_auc,NB_auc,QDA_auc,SVM_auc,number_of_features=score_model(cols, cat_cols)


    var0(t-1) var1(t-1) var2(t-1) var3(t-1) var4(t-1) var5(t-1) var6(t-1)  \
0         NaN       NaN       NaN       NaN       NaN       NaN       NaN   
1          70      2.25       3.0       3.0       3.0       3.0       1.0   
2          38       2.5       3.0       2.5       3.0       3.0       1.0   
3          38       2.5       3.0       2.5       3.0       3.0       1.0   
4          38      2.25       3.0       2.5       3.0       3.0       1.0   
..        ...       ...       ...       ...       ...       ...       ...   
734        13      2.25       3.0      2.75       3.0       3.0       1.0   
735        13       2.5       3.0       2.5       3.0       3.0       1.0   
736        13       2.0       2.0       2.5       3.0       1.0       1.0   
737        72       2.5       3.0       2.5       3.0       3.0       1.5   
738        72       2.5       3.0       2.5       3.0       3.0      1.75   

    var7(t-1) var8(t-1) var10(t-1) var11(t-1) var12(t-1) var13(t-1)  \
0   

c:\Users\catar\Anaconda3\ANACONDA_NOVO\envs\erasmus\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 2 is smaller than n_iter=10. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


AUC_avg: 0.5435620037316294
Decision Tree
Grid, 400
AUC_avg: 0.5221147540980965
Random Forest
Grid, 800
AUC_avg: 0.5510649562946267
Neural Network
Grid, 12
AUC_avg: 0.5416512037933864
AdaBoost
Grid, 4


c:\Users\catar\Anaconda3\ANACONDA_NOVO\envs\erasmus\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


AUC_avg: 0.5421349602781195
Naive Bayes
Grid, 1
AUC_avg: 0.5239101994690599
QDA
Grid, 1
AUC_avg: 0.5041313321642394
RBF SVM
Grid, 2


...

AUC_avg: 0.5383343914897313
QDA
Grid, 1


c:\Users\catar\Anaconda3\ANACONDA_NOVO\envs\erasmus\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 2 is smaller than n_iter=10. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


AUC_avg: 0.5383343914897313
RBF SVM
Grid, 2


c:\Users\catar\Anaconda3\ANACONDA_NOVO\envs\erasmus\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 2 is smaller than n_iter=10. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


AUC_avg: 0.5383343914897313


In [54]:
from PIL import Image
number_of_features=np.arange(2,34,1)
#KNN_score=[0.8085106382978723,0.8031914893617021,0.8297872340425532,0.7925531914893617,0.776595744680851,0.8351063829787234,0.6861702127659575,0.6808510638297872,0.6968085106382979,0.6595744680851063,0.7021276595744681,0.6436170212765957,0.6968085106382979,0.675531914893617,0.6914893617021277,0.6542553191489362,0.6648936170212766,0.6542553191489362,0.6117021276595744,0.6648936170212766,0.648936170212766,0.6276595744680851,0.6117021276595744,0.648936170212766,0.675531914893617,0.6436170212765957,0.6542553191489362,0.6436170212765957,0.6382978723404256,0.6542553191489362,0.6648936170212766,0.6648936170212766]
KNN_score=np.array(KNN_acc)
#DT_score=[0.4946808510638298,0.4521276595744681,0.48404255319148937,0.5212765957446809,0.7712765957446809,0.4574468085106383,0.6914893617021277,0.44148936170212766,0.7074468085106383,0.6914893617021277,0.7074468085106383,0.3882978723404255,0.7074468085106383,0.6329787234042553,0.6436170212765957,0.7074468085106383,0.6063829787234043,0.6382978723404256,0.5638297872340425,0.6329787234042553,0.7074468085106383,0.7074468085106383,0.7074468085106383,0.5904255319148937,0.7074468085106383,0.6329787234042553,0.4574468085106383,0.48936170212765956,0.7712765957446809,0.6382978723404256,0.4627659574468085,0.5691489361702128]
DT_score=np.array(DT_acc)
#RF_score=[0.48404255319148937,0.7074468085106383,0.6223404255319149,0.6861702127659575,0.675531914893617,0.6702127659574468,0.05851063829787234,0.6595744680851063,0.648936170212766,0.6542553191489362,0.723404255319149,0.6914893617021277,0.7340425531914894,0.6542553191489362,0.6648936170212766,0.6382978723404256,0.6382978723404256,0.6702127659574468,0.6702127659574468,0.6170212765957447,0.6276595744680851,0.675531914893617,0.7606382978723404,0.6808510638297872,0.5904255319148937,0.6968085106382979,0.6276595744680851,0.6808510638297872,0.6170212765957447,0.6914893617021277,0.6968085106382979,0.7074468085106383]
RF_score=np.array(RF_acc)
#NN_score=[0.48936170212765956,0.6808510638297872,0.6648936170212766,0.648936170212766,0.7925531914893617,0.7606382978723404,0.7606382978723404,0.675531914893617,0.7606382978723404,0.723404255319149,0.7446808510638298,0.7021276595744681,0.7340425531914894,0.7287234042553191,0.6968085106382979,0.7180851063829787,0.6861702127659575,0.723404255319149,0.6808510638297872,0.6914893617021277,0.7127659574468085,0.7021276595744681,0.6063829787234043,0.7180851063829787,0.723404255319149,0.7340425531914894,0.7659574468085106,0.75,0.7659574468085106,0.7553191489361702,0.75,0.7393617021276596]
NN_score=np.array(NN_acc)
#AB_score=[0.5212765957446809,0.6702127659574468,0.5053191489361702,0.5106382978723404,0.6808510638297872,0.648936170212766,0.6702127659574468,0.7021276595744681,0.6542553191489362,0.6117021276595744,0.6648936170212766,0.6436170212765957,0.75,0.6861702127659575,0.6968085106382979,0.7446808510638298,0.7393617021276596,0.7074468085106383,0.6861702127659575,0.7287234042553191,0.7180851063829787,0.7287234042553191,0.7021276595744681,0.6914893617021277,0.6914893617021277,0.723404255319149,0.7446808510638298,0.7180851063829787,0.7340425531914894,0.7287234042553191,0.723404255319149,0.7021276595744681]
AB_score=np.array(AB_acc)
#NB_score=[0.6648936170212766,0.7180851063829787,0.675531914893617,0.6702127659574468,0.675531914893617,0.6595744680851063,0.648936170212766,0.6702127659574468,0.6542553191489362,0.6436170212765957,0.6170212765957447,0.6170212765957447,0.6329787234042553,0.6382978723404256,0.6542553191489362,0.6702127659574468,0.6436170212765957,0.6648936170212766,0.6542553191489362,0.6648936170212766,0.6702127659574468,0.6170212765957447,0.6117021276595744,0.6276595744680851,0.6595744680851063,0.6808510638297872,0.6914893617021277,0.7021276595744681,0.675531914893617,0.6702127659574468,0.6648936170212766,0.6542553191489362]
NB_score=np.array(NB_acc)
#QDA_score=[0.425531914893617,0.5585106382978723,0.5372340425531915,0.5904255319148937,0.6542553191489362,0.7287234042553191,0.7127659574468085,0.7340425531914894,0.7393617021276596,0.7393617021276596,0.7021276595744681,0.7287234042553191,0.7446808510638298,0.7659574468085106,0.7978723404255319,0.7872340425531915,0.7925531914893617,0.7819148936170213,0.7606382978723404,0.7978723404255319,0.7819148936170213,0.7872340425531915,0.7819148936170213,0.7872340425531915,0.7872340425531915,0.8031914893617021,0.7925531914893617,0.7978723404255319,0.8031914893617021,0.7606382978723404,0.8031914893617021,0.7553191489361702]
QDA_score=np.array(QDA_acc)
#SVM_score=[0.48404255319148937,0.5425531914893617,0.6702127659574468,0.675531914893617,0.776595744680851,0.7393617021276596,0.75,0.7340425531914894,0.7712765957446809,0.6648936170212766,0.723404255319149,0.6808510638297872,0.6861702127659575,0.7287234042553191,0.7712765957446809,0.7553191489361702,0.75,0.7553191489361702,0.7340425531914894,0.7446808510638298,0.75,0.7446808510638298,0.75,0.7712765957446809,0.7553191489361702,0.7872340425531915,0.7872340425531915,0.7872340425531915,0.7712765957446809,0.776595744680851,0.7819148936170213,0.7606382978723404]
SVM_score=np.array(SVM_acc)

plt.plot(KNN_score, color='r', label='k nearest neighbours')
plt.plot(DT_score, color='r', label='decision tree')
plt.plot(RF_score, color='b', label='random forest')
plt.plot(NN_score, color='b', label='neural network')
plt.plot(AB_score, color='m', label='adaboost')
plt.plot(NB_score, color='c', label='naive bayes')
plt.plot(QDA_score, color='y', label='qda')
plt.plot(SVM_score, color='y', label='svm')
plt.legend()
plt.show()
plt.savefig('C:/Users/catar/Desktop/TUe/Conferences/PETRA_2023/Code_and_data/Best_scenarios/sensor_location_action_report/classifiers_standardization/classifiers_features_accuracies.png')
plt.close()

<IPython.core.display.Javascript object>